# CHROMA DATABASE


In [155]:
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
import os

In [156]:
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [157]:
# doc_loader=PyPDFLoader("pdfs/JS.pdf")
doc_loader=PyPDFDirectoryLoader("pdfs")
doc_data=doc_loader.load()

In [158]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
text_chunk=text_splitter.split_documents(doc_data)

In [159]:
llm=OpenAI(
    temperature=0.7,
    openai_api_key=OPENAI_API_KEY
)
embeddings=OpenAIEmbeddings()

In [160]:
persist_directory='./chromedb'

In [161]:
vectordb=Chroma.from_documents(
    documents=text_chunk,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [162]:
retriever  = vectordb.as_retriever(search_kwargs={"k": 4})

In [163]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever 
)

In [164]:
import sys

print("=== Q&A System ===")
print("Type 'exit' to quit\n")

while True:
    try:
        user_input = input("Enter Query: ").strip()
        
        if user_input.lower() in ["exit", "quit"]:
            print("EXITING....")
            sys.exit()
        
        # Skip empty input
        if not user_input:
            continue
            
        # Process query
        result = qa({"query": user_input})
        print(f"\nQ: {user_input}")
        print(f"A: {result['result']}")
        print("-" * 40 + "\n")
        
    except KeyboardInterrupt:
        print("\nExiting...")
        sys.exit()
    except Exception as e:
        print(f"Error: {e}\n")

=== Q&A System ===
Type 'exit' to quit


Q: what is python?
A:  Python is a widely used general-purpose, high level programming language that was created by Guido van Rossum in 1991 and further developed by the Python Software Foundation. It was designed with an emphasis on code readability and its syntax allows programmers to express their concepts in fewer lines of code. It is available and can run on various operating systems, making it a cross-platform and portable language. It is also an open source programming language with a large standard library and is free to download and use. Some features of Python include being simple and easy to learn, supporting exception handling, and having advanced features like generators and list comprehensions. However, it also has limitations in terms of performance and may not be suitable for mobile applications. There are also different "flavors" of Python, such as Cpython and Jython, that have slight variations in their implementation.
--------

SystemExit: 

C:\Users\rajun\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
